In [1]:
# # please ignore warning messages during the installation
# !pip install --upgrade sagemaker
# !pip install --disable-pip-version-check -q 
# !conda install -q -y pytorch==1.6.0 -c pytorch
# !pip install --disable-pip-version-check -q transformers==3.5.1
# !pip install -q protobuf==3.20.*

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
import boto3
import sagemaker
import pandas as pd
import botocore

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
sagemaker.__version__

'2.227.0'

In [4]:
# ========================== low-level service client of the boto3 session ==========================
config = botocore.config.Config(user_agent_extra='bedissj-1699438736259')


sm = boto3.client(service_name='sagemaker', 
                  config=config)

sm_runtime = boto3.client('sagemaker-runtime',
                          config=config)

sess = sagemaker.Session(sagemaker_client=sm,
                         sagemaker_runtime_client=sm_runtime)

bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = sess.boto_region_name

cw = boto3.client(service_name='cloudwatch', 
                  config=config)

autoscale = boto3.client(service_name="application-autoscaling", 
                         config=config)

## 1. Create an endpoint with multiple variants 

In [5]:
model_a_s3_uri = "s3://{}/y8a6c91lumhh-ModelTra-GTH766MIe3-002-1a2991b7/output/model.tar.gz".format(bucket)
model_b_s3_uri = "s3://{}/y8a6c91lumhh-ModelTra-GTH766MIe3-010-b1656de1/output/model.tar.gz".format(bucket)

### 1.1 Construct Docker Image URI

In [6]:
FRAMEWORK_VERSION = '1.0-1'
deploy_instance_type = "ml.m5.large"

In [7]:
churn_inference_image_uri = sagemaker.image_uris.retrieve(
    framework='sklearn',
    version=FRAMEWORK_VERSION,
    instance_type=deploy_instance_type,
    image_scope="inference",
    region=region
)

print(churn_inference_image_uri)

659782779980.dkr.ecr.eu-west-3.amazonaws.com/sagemaker-scikit-learn:1.0-1-cpu-py3


In [19]:
import time

timestamp = int(time.time())

model_name_a = "churn-prediction-mon1-model-a-{}".format(timestamp)
model_name_b = "churn-prediction-mon1-model-b-{}".format(timestamp)


Check if models already exist.

In [20]:
def check_model_existence(model_name):
    for model in sm.list_models()['Models']:
        if model_name == model['ModelName']:
            return True
    return False

### 1.2 Create models

In [21]:
from sagemaker.model import Model
from pprint import pprint


if not check_model_existence(model_name=model_name_a):
    model_a = Model(
        name=model_name_a,
        model_data=model_a_s3_uri,
        image_uri=churn_inference_image_uri,
        entry_point='./src/inference.py',
        sagemaker_session=sess,
        role=role
    )
    model_a.create()
    pprint(model_a)
    
else:
    print("Model {} already exists".format(model_name_a))

In [22]:
if not check_model_existence(model_name=model_name_b):
    # model_b = sm.create_model(
    #     ModelName=model_name_b,
    #     ExecutionRoleArn=role,
    #     PrimaryContainer={
    #         "ModelDataUrl": model_b_s3_uri,
    #         "Image":churn_inference_image_uri,
    #     }
    # )
    # pprint(model_b)

    model_b = Model(
        name=model_name_b,
        model_data=model_b_s3_uri,
        image_uri=churn_inference_image_uri,
        entry_point='./src/inference.py',
        sagemaker_session=sess,
        role=role
    )
    model_b.create()
    pprint(model_b)

else:
    print("Model {} already exists".format(model_name_b))

### 1.3 Create production variants

In [23]:
from sagemaker.session import production_variant

variantA = production_variant(
    model_name=model_name_a,
    instance_type=deploy_instance_type,
    initial_instance_count=1,
    initial_weight=50,
    variant_name='VariantA'
)
print(variantA)

{'VariantName': 'VariantA', 'ModelName': 'churn-prediction-mon1-model-a-1728652261', 'InitialVariantWeight': 50, 'InitialInstanceCount': 1, 'InstanceType': 'ml.m5.large'}


In [24]:
variantB = production_variant(
    model_name=model_name_b,
    instance_type=deploy_instance_type,
    initial_instance_count=1,
    initial_weight=50,
    variant_name='VariantB'
)
print(variantA)

{'VariantName': 'VariantA', 'ModelName': 'churn-prediction-mon1-model-a-1728652261', 'InitialVariantWeight': 50, 'InitialInstanceCount': 1, 'InstanceType': 'ml.m5.large'}


### 1.4 Configure and create the endpoint

Check endpoint configuration existence.

In [25]:
def check_endpoint_config_existence(endpoint_config_name):
    for endpoint_config in sm.list_endpoint_configs()['EndpointConfigs']:
        if endpoint_config_name == endpoint_config['EndpointConfigName']:
            return True
    return False


def check_endpoint_existence(endpoint_name):
    for endpoint in sm.list_endpoints()['Endpoints']:
        if endpoint_name == endpoint['EndpointName']:
            return True
    return False

Create endpoint configuration for  A/B testing.

In [26]:
endpoint_config_name = "churn-prediction-ab-epc-{}".format(timestamp)

if not check_endpoint_config_existence(endpoint_config_name):
    endpoint_config = sm.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[variantA, variantB]
    )
    pprint(endpoint_config)
else: 
     print("Endpoint configuration {} already exists".format(endpoint_config_name))

{'EndpointConfigArn': 'arn:aws:sagemaker:eu-west-3:668303144976:endpoint-config/churn-prediction-ab-epc-1728652261',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '115',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Fri, 11 Oct 2024 13:11:07 GMT',
                                      'x-amzn-requestid': '8f9e7eab-8667-4e26-8d19-bc9a45c9c1a1'},
                      'HTTPStatusCode': 200,
                      'RequestId': '8f9e7eab-8667-4e26-8d19-bc9a45c9c1a1',
                      'RetryAttempts': 0}}


In [27]:
model_ab_endpoint_name = 'churn-prediction-ab-ep-{}'.format(timestamp)
print('Endpoint name: {}'.format(model_ab_endpoint_name))

Endpoint name: churn-prediction-ab-ep-1728652261


In [28]:
if not check_endpoint_existence(model_ab_endpoint_name):
    endpoint_response = sm.create_endpoint(
        EndpointName=model_ab_endpoint_name,
        EndpointConfigName=endpoint_config_name
    )
    print('Creating endpoint {}'.format(model_ab_endpoint_name))
    pprint(endpoint_response)
else:
    print("Endpoint {} already exists".format(model_ab_endpoint_name))

Creating endpoint churn-prediction-ab-ep-1728652261
{'EndpointArn': 'arn:aws:sagemaker:eu-west-3:668303144976:endpoint/churn-prediction-ab-ep-1728652261',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '101',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Fri, 11 Oct 2024 13:11:09 GMT',
                                      'x-amzn-requestid': 'd522e65d-0761-4d73-ab9a-962482f45f82'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'd522e65d-0761-4d73-ab9a-962482f45f82',
                      'RetryAttempts': 0}}


In [29]:
from IPython.core.display import display, HTML

display(
    HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST endpoint</a></b>'.format(
        region, 
        model_ab_endpoint_name)
        )
)

/tmp/ipykernel_241/24894441.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [30]:
%%time

waiter = sm.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=model_ab_endpoint_name)

CPU times: user 33.8 ms, sys: 6.84 ms, total: 40.7 ms
Wall time: 162 ms


In [65]:
tst = pd.read_csv("./src/BankChurners_mon1 (4).csv")
tst.drop(columns=list(tst.columns[0]), inplace=True)

display(tst)

,1,56,1.1,2,6,1.2,2.1,0.1,49,3,3.1,3.2,4058.0,793,3265.0,0.758,15865,105,0.667,0.195
0,1,47,1,2,0,1,0,0,35,4,3,2,3636.0,1010,2626.0,0.596,1564,59,0.686,0.278
1,0,61,1,0,2,1,2,0,56,6,2,3,9918.0,850,9068.0,0.688,2485,40,0.290,0.086
2,1,40,1,3,3,1,1,0,33,4,3,3,6884.0,1001,5883.0,0.786,975,11,0.833,0.145
3,1,46,1,4,2,2,3,0,26,4,3,0,1791.0,1337,454.0,0.763,3592,46,0.917,0.747
4,1,33,0,2,2,2,4,0,19,3,2,3,5141.0,1737,3404.0,0.985,5859,82,0.783,0.338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020,1,43,1,1,6,2,2,0,31,4,3,1,2283.0,1470,813.0,0.935,5100,76,0.689,0.644
2021,1,38,1,2,3,2,2,3,36,5,3,2,31546.0,2057,29489.0,0.625,15311,108,0.714,0.065
2022,1,52,1,2,3,2,0,0,46,4,2,4,34516.0,0,34516.0,0.775,4037,87,0.706,0.000
2023,1,45,1,3,2,1,3,0,36,6,3,2,21630.0,0,21630.0,0.804,4584,84,0.750,0.000


In [62]:
from sagemaker.predictor import Predictor
from sagemaker.base_serializers import CSVSerializer

predictor = Predictor(
    endpoint_name=model_ab_endpoint_name, # Replace None
    sagemaker_session=sess
)


Complete prediction script.